# Demo: Utilising multiple STAC Catalogs

This notebook demonstrates the value of using `RasterBase` along with `QueryParams` and `LoadParams` to enable loading the same data from different sources with minimal changes to code.

This notebook demonstrates:

1. How to load data for Africa from Digital Earth Africa and Element 84
1. How to load data for Australia from Digital Earth Australia and Element 84

For more information on these classes, see [loading_demo.ipynb](loading_demo.ipynb).

## Set up
The following cell should be uncommented and run if you installed the package in editable mode and are actively developing and testing modules.
Otherwise, it can be left commented.

In [ ]:
# %load_ext autoreload
# %autoreload 2

### Enable logging

This will allow you to see info and warning messages from the package.

In [ ]:
import logging
import sys

logging.basicConfig(
    format="%(asctime)s | %(levelname)s : %(message)s",
    level=logging.INFO,
    stream=sys.stdout,
)

### Import the relevant packages

In [ ]:
import numpy as np

from eo_insights.band_indices import calculate_indices
from eo_insights.raster_base import RasterBase, QueryParams, LoadParams
from eo_insights.stac_configuration import (
    de_africa_stac_config,
    de_australia_stac_config,
    element_84_stac_config,
)

# Africa 
In this section we load data over Africa from DE Africa and Element 84.

### Set up area of interest and time to load
The query and load parameters can be used for both catalogs.

In [ ]:
africa_query_params = QueryParams(
    bbox=(37.76, 12.49, 37.77, 12.50),
    start_date="2020-11-01",
    end_date="2020-12-01",
)

africa_load_params = LoadParams(
    crs="EPSG:6933", resolution=20, bands=("red", "green", "blue", "nir", "scl")
)

### Load and plot data from DE Africa

In [ ]:
deafrica_stac_raster = RasterBase.from_stac_query(
    config=de_africa_stac_config,
    collections=["s2_l2a"],
    query_params=africa_query_params,
    load_params=africa_load_params,
)

In [ ]:
deafrica_stac_raster.data = calculate_indices(deafrica_stac_raster.data, index="ndvi")

deafrica_stac_raster.data.ndvi.isel(time=0).plot(cmap="RdYlGn")

### Load and plot data from Element 84

In [ ]:
element84_africa_stac_raster = RasterBase.from_stac_query(
    config=element_84_stac_config,
    collections=["sentinel-2-l2a"],
    query_params=africa_query_params,
    load_params=africa_load_params,
)

In [ ]:
element84_africa_stac_raster.data = calculate_indices(
    element84_africa_stac_raster.data, index="ndvi"
)

element84_africa_stac_raster.data.ndvi.isel(time=0).plot(cmap="RdYlGn")

# Australia 
In this section we load data over Australia from DE Australia and Element 84.

### Set up area of interest and time to load
The query parameters can be used for both catalogs.
The load parameters must be configured independently, as the two catalogs use different pixel quality bands.

In [ ]:
aus_query_params = QueryParams(
    bbox=(145.02, -37.46, 145.01, -37.45),
    start_date="2020-11-01",
    end_date="2020-12-01",
)

### Load and plot data from DE Australia

In [ ]:
deaus_load_params = LoadParams(
    crs="EPSG:3577", resolution=10, bands=("red", "green", "blue", "nir", "fmask")
)

In [ ]:
deaus_stac_raster = RasterBase.from_stac_query(
    config=de_australia_stac_config,
    collections=["ga_s2am_ard_3", "ga_s2bm_ard_3"],
    query_params=aus_query_params,
    load_params=deaus_load_params,
)

In [ ]:
deaus_stac_raster.data = calculate_indices(deaus_stac_raster.data, index="ndvi")

deaus_stac_raster.data.ndvi.isel(time=1).plot(cmap="RdYlGn")

### Load and plot data from Element 84

In [ ]:
element84_aus_load_params = LoadParams(
    crs="EPSG:3577", resolution=10, bands=("red", "green", "blue", "nir", "scl")
)

In [ ]:
element84_aus_stac_raster = RasterBase.from_stac_query(
    config=element_84_stac_config,
    collections=["sentinel-2-l2a"],
    query_params=aus_query_params,
    load_params=element84_aus_load_params,
)

In [ ]:
element84_aus_stac_raster.data = calculate_indices(
    element84_aus_stac_raster.data, index="ndvi"
)

element84_aus_stac_raster.data.ndvi.isel(time=1).plot(cmap="RdYlGn")